In [3]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py39" --user
!python3 --version

--2024-12-27 15:34:26--  https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61451533 (59M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  58.60M  98.8MB/s    in 0.6s    

2024-12-27 15:34:27 (98.8 MB/s) - ‘mini.sh’ saved [61451533/61451533]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ 

In [4]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/local/lib/python3.9 1

update-alternatives: using /usr/local/lib/python3.9 to provide /usr/bin/python3 (python3) in auto mode


In [5]:
%pip install SimpleITK==2.4.0
%pip install numpy==1.26.4
%pip install pyradiomics==3.1.0
%pip install six==1.17.0

     |████████████████████████████████| 52.4 MB 207 kB/s 
     |████████████████████████████████| 18.2 MB 1.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.3
    Uninstalling numpy-1.22.3:
      Successfully uninstalled numpy-1.22.3
     |████████████████████████████████| 177 kB 4.6 MB/s 
     |████████████████████████████████| 4.5 MB 11.7 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=8f74440889424c1321b401acb9c36b99b964c828acffbd09088c6c94c4092595
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0


In [8]:
%%writefile code.py

import radiomics
import numpy as np
import SimpleITK as sitk
import radiomics.featureextractor
import os
import six
from os import path

def featureExtractor(fileId):
  imagePath = '/content/drive/MyDrive/LuanVan/dataset/BraTS2021_Training_Data/%s/%s_flair.nii.gz' % (fileId, fileId)
  image = sitk.ReadImage(imagePath)
  maskPath ='/content/drive/MyDrive/LuanVan/dataset/BraTS2021_Training_Data/%s/%s_tumor.nii.gz' % (fileId, fileId)
  mask = sitk.ReadImage(maskPath)

  kernel = 5
  settings = {}
  settings['kernelRadius'] = kernel
  settings['voxelBatch'] = 2048
  extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(**settings)
  extractor.disableAllFeatures()
  extractor.enableFeatureClassByName('firstorder')

  featureMap = extractor.execute(image, mask, voxelBased=True)

  for featureName, featureValue in six.iteritems(featureMap):
    if isinstance(featureValue, sitk.Image):
      fileFolder = '/content/drive/MyDrive/LuanVan/dataset/kernel%s/%s' % (kernel, fileId)
      if path.exists(fileFolder) == False:
        os.mkdir(fileFolder)
      sitk.WriteImage(featureValue, '%s/%s.nrrd' % (fileFolder, featureName))
      print('Computed %s, stored as "%s/%s.nrrd"' % (featureName, fileFolder, featureName))
    else:
      print('%s: %s' % (featureName, featureValue))


datasetDir = os.path.join('/content/drive/MyDrive/LuanVan/dataset/BraTS2021_Training_Data')

listDir = os.listdir(datasetDir)
listDir.sort()
listDir
# Iterate over files in directory
listDir = os.listdir(datasetDir)
listDir.sort()
for name in listDir:
  if name.startswith('BraTS2021_'):
    print('Starting %s' % (name))
    featureExtractor(name)

Overwriting code.py


In [7]:
!python3 code.py

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Traceback (most recent call last):
  File "/content/code.py", line 19, in <module>
    featureMap = extractor.execute(image, mask, voxelBased=True)
  File "/usr/local/lib/python3.9/site-packages/radiomics/featureextractor.py", line 330, in execute
    featureVector.update(self.computeFeatures(inputImage, inputMask, imageTypeName, **inputKwargs))
  File "/usr/local/lib/python3.9/site-packages/radiomics/featureextractor.py", line 517, in computeFeatures
    for (featureName, featureValue) in six.iteritems(featureClass.execute()):
  File "/usr/local/lib/python3.9/site-packages/radiomics/base.py", line 183, in execute
    self._calculateVoxels()
  File "/usr/local/lib/python3.9/site-packages/radiomics/base.py", l